# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [6]:
import pandas as pd
import sqlite3 
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()



## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [15]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
cur.execute("""SELECT contactFirstName, contactLastName, addressLine1, creditLimit FROM customers 
                WHERE state == 'CA'
                AND creditLimit > 25000;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(10)

,contactFirstName,contactLastName,addressLine1,creditLimit
0,Susan,Nelson,5677 Strong St.,210500
1,Julie,Murphy,5557 North Pendale Street,64600
2,Juri,Hashimoto,9408 Furth Circle,84600
3,Julie,Young,78934 Hillside Dr.,90700
4,Valarie,Thompson,361 Furth Circle,105000
5,Julie,Brown,7734 Strong St.,105000
6,Brian,Chandler,6047 Douglas Av.,57700
7,Sue,Frick,3086 Ingle Ln.,77600
8,Steve,Thompson,3675 Furth Circle,55400
9,Sue,Taylor,2793 Furth Circle,60300


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [18]:
cur.execute('''SELECT state, AVG(creditLimit) FROM customers GROUP BY state;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(10)

,state,AVG(creditLimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [20]:
cur.execute('''SELECT contactFirstName,
                contactLastName, 
                orderNumber, 
                status, 
                quantityOrdered
                FROM customers 
                JOIN orders 
                USING (customerNumber) 
                JOIN orderDetails 
                USING (orderNumber);''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(10)

,contactFirstName,contactLastName,orderNumber,status,quantityOrdered
0,Carine,Schmitt,10123,Shipped,26
1,Carine,Schmitt,10123,Shipped,34
2,Carine,Schmitt,10123,Shipped,46
3,Carine,Schmitt,10123,Shipped,50
4,Carine,Schmitt,10298,Shipped,32
5,Carine,Schmitt,10298,Shipped,39
6,Carine,Schmitt,10345,Shipped,43
7,Jean,King,10124,Shipped,21
8,Jean,King,10124,Shipped,22
9,Jean,King,10124,Shipped,22


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [24]:
cur.execute('''SELECT customerName, customerNumber, productName, productCode, quantityOrdered 
            FROM customers 
            JOIN orders
            USING (customerNumber) 
            JOIN orderDetails
            USING (orderNumber)
            JOIN products 
            USING (productCode)
            WHERE quantityOrdered > 9 
            ORDER BY quantityOrdered DESC;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(10)

,customerName,customerNumber,productName,productCode,quantityOrdered
0,Mini Caravy,209,1969 Dodge Charger,S12_4675,97
1,"Down Under Souveniers, Inc",323,1969 Dodge Super Bee,S18_3278,90
2,Tekni Collectables Inc.,328,America West Airlines B757-200,S700_2466,85
3,Tekni Collectables Inc.,328,F/A 18 Hornet 1/72,S700_3167,77
4,"Down Under Souveniers, Inc",323,1970 Plymouth Hemi Cuda,S12_3990,77
5,Mini Caravy,209,1956 Porsche 356A Coupe,S24_3856,76
6,The Sharp Gifts Warehouse,450,1917 Grand Touring Sedan,S18_1749,76
7,The Sharp Gifts Warehouse,450,1949 Jaguar XK 120,S24_2766,76
8,Euro+ Shopping Channel,141,1962 Volkswagen Microbus,S24_2300,70
9,Salzburg Collectables,382,1956 Porsche 356A Coupe,S24_3856,70


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [31]:
cur.execute('''SELECT lastName, firstName, employeeNumber, officeCode
                    FROM employees
                    WHERE officeCode IN 
                    (SELECT officeCode 
                        FROM offices 
                        JOIN employees
                        USING(officeCode)
                        GROUP BY officeCode
                        HAVING COUNT(employeeNumber) < 5);''')
            
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(10)

,lastName,firstName,employeeNumber,officeCode
0,Patterson,William,1088,6
1,Firrelli,Julie,1188,2
2,Patterson,Steve,1216,2
3,Tseng,Foon Yue,1286,3
4,Vanauf,George,1323,3
5,Bott,Larry,1501,7
6,Jones,Barry,1504,7
7,Fixter,Andy,1611,6
8,Marsh,Peter,1612,6
9,King,Tom,1619,6


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!